In [1]:
import numpy as np
import sklearn
import pandas as pd
import scipy
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import normalize
from sklearn import linear_model
from sklearn.cross_validation import StratifiedKFold
from sklearn.linear_model import LogisticRegression
import pickle
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import accuracy_score

# load samples

In [4]:
X=pd.read_csv("X.csv",header=None)
Y=pd.read_csv("Y.csv",header=None)

x =np.asarray(X)
Y_arr = np.asarray(Y)

# normalize
x_normed = (x - x.mean(axis=0))/x.std(axis=0)
x_normed=(x_normed)

# Under sample it

# combine x and y
xy=np.concatenate((x_normed,Y_arr),axis=1)
df=pd.DataFrame(xy) # the last column is label

# seperate y=0 and y=1 rows
negatives=df[df[df.columns[36]]==0.0]
positives=df[df[df.columns[36]]==1.0]
print "positives ",positives.shape,negatives.shape
random_negatives=negatives.sample(frac=0.2,random_state=11L)
print "negatives ",random_negatives.shape
sampled=np.concatenate((np.asarray(positives),np.asarray(random_negatives)),axis=0)
x_normed=sampled[:,0:36]
Y_arr=sampled[:,36]
print sampled.shape,x_normed.shape,Y_arr.shape

positives  (1600, 37) (8397, 37)
negatives  (1679, 37)
(3279, 37) (3279, 36) (3279,)


# Correlation matrix

In [5]:
plt.pcolor(np.abs(np.corrcoef(x_normed.T)))
plt.colorbar()
plt.xlim(0, 36)
plt.ylim(0,36)
plt.show()

# find most relevant features!

In [6]:
# from http://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html#example-ensemble-plot-forest-importances-py
from sklearn.ensemble import ExtraTreesClassifier
forest = ExtraTreesClassifier(n_estimators=250,random_state=0)
forest.fit(x_normed, Y_arr)
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

#for f in range(x_normed.shape[1]):
#    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(x_normed.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(x_normed.shape[1]), indices)
plt.xlim([-1, x_normed.shape[1]])
plt.show()

Feature ranking:


In [7]:
# Reduce features
x_normed=np.delete(x_normed,[14,15],axis=1)

# logistic regression and decision trees

In [13]:
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn import tree

print x_normed.shape
x_train, x_test, y_train, y_test = train_test_split(x_normed, Y_arr, test_size=0.3)
k_fold = StratifiedKFold(y_train.ravel(), n_folds=10)

auc=[[],[],[]]
odds=[]
counter=0
times=10
for i in range(times):
    for train_indx, test_indx in k_fold:
        val_train_x, val_train_y =x_train[train_indx], y_train[train_indx]
        val_test_x, val_test_y = x_train[test_indx], y_train[test_indx]
    
        est = [LogisticRegression(class_weight='balanced',penalty='l1', C=10, max_iter=500, fit_intercept=True),
        #GaussianNB(),
            AdaBoostClassifier(n_estimators=100),
            RandomForestClassifier(n_estimators=500,class_weight='balanced'\
                   ,max_depth=6,bootstrap=False\
                   ,max_features=8,criterion='entropy')]
    
        for i in range(len(est)):
            est[i].fit(val_train_x, val_train_y.ravel().T)
            if i==0:
                coefs= est[i].coef_
            #print np.exp(coefs[:,[30,31,32,33]])
            score = roc_auc_score(val_test_y, est[i].predict(val_test_x))
            #print score
            auc[i].append(score)

means=[]
errs=[]
for score in auc:
    means.append(np.mean(score)) #,np.mean(auc1),np.mean(auc2),np.mean(auc3)
    errs.append(np.std(score))

print means,errs

(3279, 34)
[0.63910802342008721, 0.64211392848481585, 0.65163449478404611] [0.042939514681446009, 0.029643126857972518, 0.034883694076084472]


# Make bar plot

In [14]:
means=[]
errs=[]
for score in auc:
    means.append(np.mean(score)) #,np.mean(auc1),np.mean(auc2),np.mean(auc3)
    errs.append(np.std(score))

print means,errs
# Plot the feature importances of the forest
plt.figure()
plt.title("Model performance")
#plt.bar(np.arange(4),height=1,color="r", data=means,yerr=errs, align="center")
#plt.xticks(range(x_normed.shape[1]), indices)
N = 3

ind = np.arange(N)    # the x locations for the groups
width = 0.35       # the width of the bars: can also be len(x) sequence

p1 = plt.bar(ind, means, width=0.6, color='#ffff00', yerr=errs)
plt.axhline(y=0.5)
plt.xlim(-0.1, 2.8)
plt.show()

[0.63910802342008721, 0.64211392848481585, 0.65163449478404611] [0.042939514681446009, 0.029643126857972518, 0.034883694076084472]


# optimize random forest

In [8]:
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn import tree

print x_normed.shape
x_train, x_test, y_train, y_test = train_test_split(x_normed, Y_arr, test_size=0.3)
#print "train/validation",x_train.shape,y_train.shape
#print "test:",x_test.shape,y_test.shape

k_fold = StratifiedKFold(y_train.ravel(), n_folds=10)

est = [rf(n_estimators=50,class_weight='balanced',max_depth=5,min_samples_split=10),
            rf(class_weight='balanced',max_depth=5),
            rf(n_estimators=200,class_weight='balanced',\
                                   max_depth=6),
            rf(n_estimators=500,class_weight='balanced',max_depth=6,max_features='log2')]
auc=[[] for i in est]
odds=[]
counter=0
times=1

for i in range(times):
    for train_indx, test_indx in k_fold:
        val_train_x, val_train_y =x_train[train_indx], y_train[train_indx]
        val_test_x, val_test_y = x_train[test_indx], y_train[test_indx]
        
        for i in range(len(est)):
            est[i].fit(val_train_x, val_train_y.ravel().T)
            score = roc_auc_score(val_test_y, est[i].predict(val_test_x))
            #print score
            auc[i].append(score)

means=[]
errs=[]
for score in auc:
    means.append(np.mean(score)) #,np.mean(auc1),np.mean(auc2),np.mean(auc3)
    errs.append(np.std(score))

print means#,errs

(3279, 36)
[0.60639271856245924, 0.57433835478763196, 0.60006291854808802, 0.59471853904181926]


# optimize logistic regression

In [12]:
from scipy.stats import randint as sp_randint
from sklearn.grid_search import GridSearchCV,RandomizedSearchCV
from sklearn.linear_model import LogisticRegression as lr
from operator import itemgetter

clf=lr(class_weight='balanced')

x_train, x_test, y_train, y_test = train_test_split(x_normed, Y_arr, test_size=0.3)
print x_train.shape
def report(grid_scores,n_top=5):
    top_scores=sorted(grid_scores,key=itemgetter(1),reverse=True)[:n_top]
    for i,score in enumerate(top_scores):
        print "rank",i+1
        print "mean val score",score.mean_validation_score,np.std(score.cv_validation_scores)
        print "paramters",score.parameters

param_list={"penalty":['l1','l2'],
            "C": [1,0.1,10,50,100],
            "max_iter": [200,100],
           "fit_intercept":[True,False]}
n_iter_search=15
random_search=RandomizedSearchCV(clf,param_distributions=param_list,\
                              n_iter=n_iter_search)
random_search.fit(x_train,y_train)
report(random_search.grid_scores_)

(2295, 34)
rank 1
mean val score 0.647494553377 0.00802020014059
paramters {'penalty': 'l1', 'C': 10, 'max_iter': 200, 'fit_intercept': True}
rank 2
mean val score 0.647058823529 0.0111876429783
paramters {'penalty': 'l1', 'C': 1, 'max_iter': 200, 'fit_intercept': True}
rank 3
mean val score 0.647058823529 0.0084066864836
paramters {'penalty': 'l2', 'C': 100, 'max_iter': 200, 'fit_intercept': True}
rank 4
mean val score 0.647058823529 0.0084066864836
paramters {'penalty': 'l2', 'C': 100, 'max_iter': 100, 'fit_intercept': True}
rank 5
mean val score 0.647058823529 0.0084066864836
paramters {'penalty': 'l1', 'C': 100, 'max_iter': 100, 'fit_intercept': True}


# grid search, random forest
http://scikit-learn.org/stable/auto_examples/model_selection/randomized_search.html

In [10]:
from scipy.stats import randint as sp_randint
from sklearn.grid_search import GridSearchCV,RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier as rf
from operator import itemgetter
clf=rf(n_estimators=200,class_weight='balanced')

x_train, x_test, y_train, y_test = train_test_split(x_normed, Y_arr, test_size=0.3)
print x_train.shape
def report(grid_scores,n_top=5):
    top_scores=sorted(grid_scores,key=itemgetter(1),reverse=True)[:n_top]
    for i,score in enumerate(top_scores):
        print "rank",i+1
        print "mean val score",score.mean_validation_score,np.std(score.cv_validation_scores)
        print "paramters",score.parameters

param_list={"max_depth":[3,4,5,6,7,8],
            "max_features": sp_randint(4, 34),
           "bootstrap":[True,False],
           "criterion":["gini","entropy"]}
n_iter_search=15
random_search=RandomizedSearchCV(clf,param_distributions=param_list,\
                              n_iter=n_iter_search)
random_search.fit(x_train,y_train)
report(random_search.grid_scores_)

(2295, 34)
rank 1
mean val score 0.653159041394 0.0114009135963
paramters {'max_features': 8, 'bootstrap': False, 'criterion': 'entropy', 'max_depth': 8}
rank 2
mean val score 0.648801742919 0.00996619386773
paramters {'max_features': 6, 'bootstrap': True, 'criterion': 'entropy', 'max_depth': 4}
rank 3
mean val score 0.647930283224 0.00792497947808
paramters {'max_features': 22, 'bootstrap': True, 'criterion': 'gini', 'max_depth': 4}
rank 4
mean val score 0.647494553377 0.00954103130377
paramters {'max_features': 22, 'bootstrap': False, 'criterion': 'entropy', 'max_depth': 4}
rank 5
mean val score 0.646623093682 0.01087248289
paramters {'max_features': 10, 'bootstrap': False, 'criterion': 'entropy', 'max_depth': 7}



# test sample plot

In [18]:
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import normalize
from sklearn import linear_model
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import pickle
from sklearn import metrics
from sklearn.metrics import roc_curve, auc, roc_auc_score, accuracy_score

x_train, x_test, y_train, y_test = train_test_split(x_normed, Y_arr, test_size=0.3)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3)

print x_train.shape,y_train.shape
print x_val.shape,y_val.shape
print x_test.shape,y_test.shape

est = LogisticRegression(class_weight='balanced')
est.fit(x_train, y_train.ravel())
false_positive_rate, true_positive_rate, thresholds = \
    roc_curve(y_test, est.predict(x_test))
print "false pos ", false_positive_rate    
print auc(false_positive_rate, true_positive_rate)
roc_auc = auc(false_positive_rate, true_positive_rate)
print roc_auc_score(y_test, est.predict(x_test))
plt.plot(false_positive_rate, true_positive_rate, 'b',
label='Test AUC = %0.2f'% roc_auc,color='green')
false_positive_rate, true_positive_rate, thresholds = \
    roc_curve(y_val, est.predict(x_val))

print auc(false_positive_rate, true_positive_rate)
roc_auc = auc(false_positive_rate, true_positive_rate)
print roc_auc_score(y_val, est.predict(x_val))

#plt.plot(false_positive_rate, true_positive_rate, 'b',
#label='Validation AUC = %0.2f'% roc_auc)
plt.plot([0,1],[0,1],'r--')
with open('model.output','wb') as myfile:
    pickle.dump(est,myfile)
legend = plt.legend(loc='upper center', shadow=True)    
plt.show()

(1606, 34) (1606,)
(689, 34) (689,)
(984, 34) (984,)
false pos  [ 0.         0.3253493  1.       ]
0.635461995264
0.635461995264
0.628237861705
0.628237861705


working logistic regression model